In [1]:
import mysql.connector as conn
import pandas as pd
db = conn.connect(
    host='localhost',
    user='root',
    password='123456789zxcvbnm',
    database = 'shop',
)

# 创建游标对象，以便执行 SQL 语句
cursor = db.cursor(buffered=True)

In [2]:
### 查看 products 表
select_all = "SELECT * FROM products"
cursor.execute(select_all)

# 获取查询结果
res = cursor.fetchall()

# 将结果转换
df_data1 = pd.DataFrame(res, columns=cursor.column_names)
df_data1

,product_id,product_name,product_type,sale_price,purchase_price,regist_date
0,0001,T恤衫,衣服,1000,500.0,2009-09-20
1,0002,打孔器,办公用品,500,320.0,2009-09-11
2,0003,运动T恤,衣服,4000,2800.0,None
3,0004,菜刀,厨房用具,3000,2800.0,2009-09-20
4,0005,高压锅,厨房用具,6800,5000.0,2009-01-15
5,0006,叉子,厨房用具,500,NaN,2009-09-20
6,0007,擦菜板,厨房用具,880,790.0,2008-04-28
7,0008,圆珠笔,办公用品,100,NaN,2009-11-11


In [6]:
#### 练习 3.1
# 创建视图
creat_view = '''CREATE VIEW ViewPractice5_5 (product_name, sale_price, regist_date)
                AS
                SELECT product_name, sale_price, regist_date
                FROM products
                WHERE sale_price >= 1000
                 AND regist_date = '2009-09-20'; '''
cursor.execute(creat_view)
db.commit()

# 查询视图中的结果
query_view = '''SELECT * FROM ViewPractice5_5'''
cursor.execute(query_view)

res = cursor.fetchall()
print(res)

df_view = pd.DataFrame(res, columns=cursor.column_names)
df_view


[('T恤衫', 1000, datetime.date(2009, 9, 20)), ('菜刀', 3000, datetime.date(2009, 9, 20))]


,product_name,sale_price,regist_date
0,T恤衫,1000,2009-09-20
1,菜刀,3000,2009-09-20


In [7]:
#### 练习3.2
insert_value_ = '''INSERT INTO ViewPractice5_5 VALUES (' 刀子 ', 300, '2009-11-02'); '''
cursor.execute(insert_value_)
db.commit()

res = cursor.fetchall()
print(res)
df_insert = pd.DataFrame(res, columns=cursor.column_names)
df_insert

DatabaseError: 1423 (HY000): Field of view 'shop.viewpractice5_5' underlying table doesn't have a default value

视图插入数据时，原表也会相应插入数据。然而：     
在 MySQL 中，可以对视图进行插入操作，但需要满足以下条件：       
  * 视图必须包含基础表中的所有非 NULL 列。
  * 视图中的列必须来自单个基础表。
  * 视图不能包含计算列、聚合函数、DISTINCT、GROUP BY、ORDER BY 等。    
由报错可知：视图 ViewPractice5_5 可能没有包括基础表 products 的所有非 NULL 列或没有默认值的列，因此插入操作失败。

In [8]:
#### 3.3 
select_extend  = '''SELECT product_id,
                           product_name,
                           product_type,
                           sale_price,
                           (SELECT AVG(sale_price)
                            FROM products AS p2
                            WHERE p1.product_id = p2.product_id) AS sale_price_avg
                    FROM products AS p1'''
cursor.execute(select_extend)

res = cursor.fetchall()
print(res)
df_selc_ext = pd.DataFrame(res, columns=cursor.column_names)
df_selc_ext


[('0001', 'T恤衫', '衣服', 1000, Decimal('1000.0000')), ('0002', '打孔器', '办公用品', 500, Decimal('500.0000')), ('0003', '运动T恤', '衣服', 4000, Decimal('4000.0000')), ('0004', '菜刀', '厨房用具', 3000, Decimal('3000.0000')), ('0005', '高压锅', '厨房用具', 6800, Decimal('6800.0000')), ('0006', '叉子', '厨房用具', 500, Decimal('500.0000')), ('0007', '擦菜板', '厨房用具', 880, Decimal('880.0000')), ('0008', '圆珠笔', '办公用品', 100, Decimal('100.0000'))]


,product_id,product_name,product_type,sale_price,sale_price_avg
0,0001,T恤衫,衣服,1000,1000.0000
1,0002,打孔器,办公用品,500,500.0000
2,0003,运动T恤,衣服,4000,4000.0000
3,0004,菜刀,厨房用具,3000,3000.0000
4,0005,高压锅,厨房用具,6800,6800.0000
5,0006,叉子,厨房用具,500,500.0000
6,0007,擦菜板,厨房用具,880,880.0000
7,0008,圆珠笔,办公用品,100,100.0000


In [9]:
#### 3.4
''' 
1. 先创建视图
2. 根据主查询讯结果匹配product_type，获取子查询结果
3. 将子查询结果再与主查询结合执行完整的SQL语句
'''
create_view_ = '''CREATE VIEW AvgPriceByType2 (product_id, product_name, product_type, sale_price, sale_price_avg_type)
                  AS
                  SELECT product_id,
                         product_name,
                         product_type,
                         sale_price,
                         (SELECT AVG(sale_price) 
                          FROM products AS p2
                          WHERE p1.product_type = p2.product_type
                          GROUP BY p2.product_type) AS sale_price_avg_type
                    FROM products AS p1'''

cursor.execute(create_view_)
db.commit()

select_view = '''SELECT * FROM AvgPriceByType2'''
cursor.execute(select_view)

res = cursor.fetchall()
df_data2 = pd.DataFrame(res, columns=cursor.column_names)
df_data2


,product_id,product_name,product_type,sale_price,sale_price_avg_type
0,0001,T恤衫,衣服,1000,2500.0000
1,0002,打孔器,办公用品,500,300.0000
2,0003,运动T恤,衣服,4000,2500.0000
3,0004,菜刀,厨房用具,3000,2795.0000
4,0005,高压锅,厨房用具,6800,2795.0000
5,0006,叉子,厨房用具,500,2795.0000
6,0007,擦菜板,厨房用具,880,2795.0000
7,0008,圆珠笔,办公用品,100,300.0000


###### 3.5
问：四则运算中含有 NULL 时（不进行特殊处理的情况下），运算结果是否必然会变为NULL ？     
答：对

In [11]:
#### 3.6
select_in_ = '''SELECT product_name, purchase_price
                FROM products
                WHERE purchase_price NOT IN (500, 2800, 5000); '''

cursor.execute(select_in_)

res = cursor.fetchall()
print(res)
df_selc_in = pd.DataFrame(res, columns=cursor.column_names)
df_selc_in

[('打孔器', 320), ('擦菜板', 790)]


,product_name,purchase_price
0,打孔器,320
1,擦菜板,790


从 products 表中选择商品的 product_name, purchase_price，其中的限制条件时 purchase_price 不等于 (500, 2800, 5000)。
而且我们看到，purchase_price 中有 ```NULL```值，但是df_selc_in 中并没有出现，原因：    
在 SQL 中，使用 NOT IN 来筛选某列中不包含指定值的行时，如果列中包含 NULL 值，则 NOT IN 会将 NULL 视为未知值，不会将其与指定值进行比较，也不会将其视为满足条件。因此，NOT IN 语句不会选择包含 NULL 值的行。

In [12]:
df_data1

,product_id,product_name,product_type,sale_price,purchase_price,regist_date
0,0001,T恤衫,衣服,1000,500.0,2009-09-20
1,0002,打孔器,办公用品,500,320.0,2009-09-11
2,0003,运动T恤,衣服,4000,2800.0,None
3,0004,菜刀,厨房用具,3000,2800.0,2009-09-20
4,0005,高压锅,厨房用具,6800,5000.0,2009-01-15
5,0006,叉子,厨房用具,500,NaN,2009-09-20
6,0007,擦菜板,厨房用具,880,790.0,2008-04-28
7,0008,圆珠笔,办公用品,100,NaN,2009-11-11


In [14]:
select_in__ = '''SELECT product_name, purchase_price
                 FROM products
                 WHERE purchase_price NOT IN (500, 2800, 5000, NULL);'''

cursor.execute(select_in__)

res = cursor.fetchall()
print(res)
df_selc_in_ = pd.DataFrame(res, columns=cursor.column_names)
df_selc_in_

[]


,product_name,purchase_price


返回为空。原因：    
在 SQL 中，使用 NOT IN 运算符时，如果列表中包含 NULL 值，则整个条件的结果可能不符合预期。NOT IN 运算符会将 NULL 视为未知值，并不会将其与其他值进行比较，也不会将其视为满足条件。因此，在使用 NOT IN 时，如果列表中包含 NULL 值，它将导致整个条件的结果为空，因为对于 SQL 引擎来说，无法确定 NULL 值是否等于给定的任何值，因此无法确定哪些行应该被排除。

In [17]:
#### 3.7
''' 
搜索 CASE 实现列方向上的聚合：
聚合函数 + CASE WHEN 表达式即可实现该效果
'''
clus_col = '''SELECT SUM(CASE WHEN sale_price <= 1000 THEN 1 ELSE 0 END) AS low_price,
                     SUM(CASE WHEN sale_price BETWEEN 1001 AND 3000 THEN 1 ELSE 0 END) AS mid_price,
                     SUM(CASE WHEN sale_price >= 3001 THEN 1 ELSE 0 END) AS high_price
              FROM products; '''
cursor.execute(clus_col)

res = cursor.fetchall()
print(res)

df_clus_col = pd.DataFrame(res, columns=cursor.column_names)
df_clus_col


[(Decimal('5'), Decimal('1'), Decimal('2'))]


,low_price,mid_price,high_price
0,5,1,2
